In [ ]:
import os
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
os.chdir('/home2/s439906/project/CASP16/oligomer/')



model = "best"
mode = "all"
phase="1"
pipeline = "ost"
pipeline = "pairwise"
parameters = [["best", "all", "0"],
                ["best", "all", "1"],
                ["best", "all", "2"],
                ["first",  "all", "0"],
                ["first",  "all", "1"],
                ["first",  "all", "2"],
                ["sixth",  "all", "1"]
                ]

for parameter in parameters:
    model = parameter[0]
    mode = parameter[1]
    phase = parameter[2]
    if pipeline == "ost":
        file_dir = "/data/data1/conglab/qcong/CASP16/stage2_oligomer_ranks/step3o"
        if phase == "0":
            files = [file for file in os.listdir(file_dir) if file.endswith(".txt") and ( "T0" in file or "H0" in file)]
            out_dir = "./score_T0_o/"
        elif phase == "1":
            files = [file for file in os.listdir(file_dir) if
                        file.endswith(".txt") and ("T1" in file or "H1" in file)]
            out_dir = "./score_T1_o/"
        elif phase == "2":
            files = [file for file in os.listdir(file_dir) if
                        file.endswith(".txt") and ("T2" in file or "H2" in file)]
            out_dir = "./score_T2_o/"
    elif pipeline == "pairwise":
        file_dir = "/data/data1/conglab/qcong/CASP16/stage2_oligomer_ranks/step3n"
        if phase == "0":
            files = [file for file in os.listdir(file_dir) if file.endswith(".txt") and ( "T0" in file or "H0" in file)]
            out_dir = "./score_T0_n/"
        elif phase == "1":
            files = [file for file in os.listdir(file_dir) if
                        file.endswith(".txt") and ("T1" in file or "H1" in file)]
            out_dir = "./score_T1_n/"
        elif phase == "2":
            files = [file for file in os.listdir(file_dir) if
                        file.endswith(".txt") and ("T2" in file or "H2" in file)]
            out_dir = "./score_T2_n/"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    files.sort()
    len(files)
    # columns model ips	ics	qsbest	dockq	lddt	tm
    scores = ["ips", "ics", "qsbest", "dockq", "lddt", "tm"]
    for score in scores:
        data_all = pd.DataFrame()
        keep_columns = ["model", score]
        for file in files:
            data = pd.read_csv(os.path.join(file_dir, file), sep="\t")
            data = data[keep_columns]
            data.set_index("model", inplace=True)
            data.index = data.index.str.extract(
                r'(\w+)TS(\w+)_(\w+)').apply(lambda x: (f"{x[0]}", f"TS{x[1]}", x[2][0]), axis=1)
            data.index = pd.MultiIndex.from_tuples(
                data.index, names=['target', 'group', 'model'])
            if model == "best":
                data = data.loc[(slice(None), slice(None), [
                    "1", "2", "3", "4", "5"]), :]
            elif model == "first":
                data = data.loc[(slice(None), slice(None),
                                        "1"), :]
            elif model == "sixth":
                data = data.loc[(slice(None), slice(None),
                                        "6"), :]
            grouped = data.groupby(["group"])
            grouped = pd.DataFrame(grouped[score].max())
            grouped.columns = [file.split(".")[0]]
            data_all = pd.concat([data_all, grouped], axis=1)
        output_file = os.path.join(out_dir, f"{score}-{model}-{mode}-raw.csv")
        print(data_all.shape)
        data_all.to_csv(output_file)

                            